In [ ]:
# import ftplib
# # ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC1_fitted_dose_response_25Feb20.xlsx
# # ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC2_fitted_dose_response_25Feb20.xlsx
# # ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC1_public_raw_data_25Feb20.csv
# # ftp://ftp.sanger.ac.uk/pub/project/cancerrxgene/releases/current_release/GDSC2_public_raw_data_25Feb20.csv


# with ftplib.FTP("ftp.sanger.ac.uk") as ftp:
#     ftp.login()
#     ftp.cwd("/pub/project/cancerrxgene/releases/current_release")
#     with open("file_1", "wb") as fp:
#         ftp.retrbinary("RETR GDSC1_fitted_dose_response_25Feb20.xlsx", fp.write)
#     with open("file_2", "wb") as fp:
#         ftp.retrbinary("RETR GDSC2_fitted_dose_response_25Feb20.xlsx", fp.write)
#     with open("file_3", "wb") as fp:
#         ftp.retrbinary("RETR GDSC1_public_raw_data_25Feb20.csv", fp.write)
#     with open("file_4", "wb") as fp:
#         ftp.retrbinary("RETR GDSC2_public_raw_data_25Feb20.csv", fp.write)
    

In [9]:
from therapy.query import QueryHandler
import pandas as pd

q = QueryHandler()

***Using Therapy Database Endpoint: http://localhost:8000***


In [26]:
import csv

gdsc_1_drugs = csv.DictReader(open("gdsc1_drugs.csv", "r"))
gdsc_2_drugs = csv.DictReader(open("gdsc2_drugs.csv", "r"))

normalized_dict = {}

total_compounds = 0
normalized_compounds = 0
for dataset in (gdsc_1_drugs, gdsc_2_drugs):
    for row in dataset:
        total_compounds += 1
        normalized = False
        if q.normalize(row["drug_name"]).match_type > 0:
            normalized = True
        for synonym in row["synonyms"].split(","):
            if q.normalize(synonym).match_type > 0:
                normalized = True
        if normalized:
            normalized_compounds += 1
            normalized_dict[int(row["drug_id"])] = True
        else:
            normalized_dict[int(row["drug_id"])] = False

print(f"Total compounds: {total_compounds}")
print(f"Normalization rate: {normalized_compounds / total_compounds}")

Total compounds: 565
Normalization rate: 0.7557522123893805


In [19]:
import pandas as pd
gdsc_1 = pd.read_excel("gdsc1.xlsx")
gdsc_2 = pd.read_excel("gdsc2.xlsx")

In [34]:
sensitivity_responses = 0
normalized_responses = 0
for df in (gdsc_1, gdsc_2):
    for _, row in df.iterrows():
        sensitivity_responses += 1
        if normalized_dict[row["DRUG_ID"]]:
            normalized_responses += 1

print(f"Total measurements: {sensitivity_responses}")
print(f"Normalization rate: {normalized_responses / sensitivity_responses}")

Total measurements: 446146
Normalization rate: 0.7441510178282446
